This is a script for calculating playoff probabilities in a fantasy sports league that is (1) points-based and (2) head-to-head. In this latest update, it no longer requires an outside excel document with information (although to save time, you may want to save the schedule dataframe externally as opposed to scraping that every time you run the script).

You'll need to replace a few references to URLs with your own league + install selenium to use this. That may be straightforward (just install the requisite packages and let the machine do the rest) but this isn't my first project using this package.

If your points system has a high probability for ties or a complex tiebreaker system you may want to augment this code to include that possibility. I treat ties as 0.5 wins.

I'm leaving in the URLs and team names, there's no PII, I'm not telling you which team is mine, and none of the names are even offensive, so I'm not super worried about privacy in this context.

**5.0 Update Notes (12.25.22)**

1. Massively overhauled structure of "does the deal" loops so that we're not appending to pandas until the end
2. Web scraping integration of schedule-reading and score-scraping, no separate manual file required now (also now resilient to team name changes)

**5.1 Update Notes (8.25.23)**

1. Looks like selenium went and cleaned itself up, code reflects that
2. FINALLY handling logins in the selenium code instead of handling manually in browser
3. The latest league I'm doing this for is 14 teams so misc changes to handle that

**6.0 Update Notes (10.31.25)**

1. Switched from web scraping (which has gotten increasingly difficult with site updates) to API
2. Major consequence: notebook must be run weekly to preserve week-by-week scoring, otherwise would need to be updated manually
3. Moved some functions to separate .py script

The next step would be to move the actual simulation script elsewhere, but it's not a function as of now so would require some additional lift that would really only make the code more readable, not more functional.

In [1]:
league_url = 'https://football.fantasysports.yahoo.com/f1/124506'
playoffs_start_wk = 15

## Package Imports & Selenium Init

In [34]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import collections
from functools import reduce

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime

import yahoo_fantasy_api
from yahoo_oauth import OAuth2
import json
import pprint

import zg_fantasy_calculator as fc

import re

pd.set_option('display.max_columns', 30)

In [3]:
credentials = pd.read_json('credentials.json')

In [5]:
#creds = {'consumer_key': credentials['yahoo_api']['client_id'], 'consumer_secret': credentials['yahoo_api']['client_secret']}

#with open('ouath2.json', "w") as f:
#   f.write(json.dumps(creds))
    
oauth = OAuth2(None, None, from_file='ouath2.json')
gm = yahoo_fantasy_api.Game(oauth, 'nfl') 
lg = gm.to_league('461.l.124506') #must be updated each season

[2025-11-04 11:44:17,193 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2025-11-04 11:44:17,195 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3.7263944149017334
[2025-11-04 11:44:17,195 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [6]:
wk_count = lg.current_week() - 1

print('Current Week: ' + str(lg.current_week()))
print('Most Recent Week: ' + str(wk_count))

Current Week: 10
Most Recent Week: 9


In [7]:
current_oppPoints = pd.DataFrame(columns=['Team', 'Rank', 'W', 'L', 'T', 'PF', 'PA'])

j = 0
for i in lg.standings():
    data = lg.standings()[j]
    row = {'Team': data['name'], 'Rank': int(data['rank']), 'W': int(data['outcome_totals']['wins']), \
            'L': int(data['outcome_totals']['losses']), 'T': int(data['outcome_totals']['ties']), \
            'PF': float(data['points_for']), 'PA': float(data['points_against'])}
    current_oppPoints.loc[len(current_oppPoints)] = row
    j += 1

current_oppPoints = current_oppPoints.set_index('Team')

In [8]:
current_oppPoints.to_csv('weekly_standings' + str(wk_count) + '.csv')

In [9]:
current_oppPoints

,Rank,W,L,T,PF,PA
Team,,,,,,
The Abe W Memorial Champions,1,7,2,0,1021.02,971.84
The Virtue Extollers,2,7,2,0,962.96,895.98
The Dream Team,3,6,3,0,974.62,852.16
Chip Kelly,4,5,4,0,1057.54,957.10
Kurtosis Warner,5,5,4,0,991.34,1022.56
Run CMC,6,5,4,0,950.34,984.52
Commish-ion Impossible,7,5,4,0,929.76,893.06
ACE of L.A.D.,8,4,5,0,937.72,949.84
Rod the Great's Team,9,4,5,0,894.06,941.58


## Setup Functions

In [10]:
#probably need to be manual now with the death of scraping; api does not provide this info

def schedule_generator():
    #run once at start of season to get schedule
    #element = driver.find_element(By.ID, 'yui_3_18_1_2_1725893424521_1668') #must be replaced each time lol
    #element = driver.find_element(By.ID, 'F-icon Fz-lg')

    service = Service('chromedriver')
    driver = webdriver.Chrome()
    driver.get(league_url)

    driver.get(league_url)

    login = driver.find_element(By.XPATH, '//*[@id="login-username"]')
    login.send_keys(credentials['yahoo']['username'])
    
    clickable = driver.find_element(By.ID, 'login-signin')
    clickable.click()
    
    time.sleep(3)
    
    pw = driver.find_element(By.XPATH, '//*[@id="login-passwd"]')
    pw.send_keys(credentials['yahoo']['password'])
    
    clickable = driver.find_element(By.ID, 'login-signin')
    clickable.click()
    
    driver.set_page_load_timeout(5)
    
    element = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section[1]/div/div/div[3]/section[1]/header/div/span/a[3]/span')
    sched = pd.DataFrame(columns=['Week', 'Team', 'Opponent'])
    for k in range(1,15): #number of weeks in reg season less one
        if k != 1:
            
            element.click()
            time.sleep(1.5)
        for i in range(1,8): #number of teams divided by two plus one (12/6 + 1 = 7)
                x1 = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section[1]/div/div/div[3]/section[1]/div/section/ul/li[" + str(i) + "]/div[2]/div[1]/div/div[1]/div/div/div[1]/a").text
                x2 = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section[1]/div/div/div[3]/section[1]/div/section/ul/li[" + str(i) + "]/div[2]/div[3]/div/div[2]/div/div/div[1]/a").text
                newrow = {'Week': k, 'Team': x1, 'Opponent': x2}

                sched = pd.concat([sched, pd.DataFrame([newrow])], ignore_index=True)
    
    #sched.to_csv('OneUSAC2025_sched.csv')
    return 0

#schedule_generator()



In [11]:
# CHECK NUMBERS AND REVISE

avg = 111.25 #based on last year, bye-adjusted
wobble = 26 #based on bye-adjusted final scores from last year, adjusted up slightly

# estimated chance of victory given difference in projected scores, based on above 

for k in range(0, 51, 5):
    j = 0
    for i in range(0,100000):
        x = np.random.normal(loc=avg+k, scale=wobble)
        y = np.random.normal(loc=avg, scale=wobble)
        if x > y:
            j = j + 1
    print(k, ' ', j/100000)

0   0.4996
5   0.55581
10   0.60838
15   0.65797
20   0.70839
25   0.75039
30   0.79433
35   0.83144
40   0.8607
45   0.88748
50   0.91343


## Score Frame + Seed Table

In [12]:
scores = pd.read_csv('scores' + str(wk_count - 1) + '.csv')

scores = scores.set_index('Team')

In [13]:
seed_table = current_oppPoints[['PF', 'W']]
seed_table.columns = ['Points', 'W']

seed_table

,Points,W
Team,,
The Abe W Memorial Champions,1021.02,7
The Virtue Extollers,962.96,7
The Dream Team,974.62,6
Chip Kelly,1057.54,5
Kurtosis Warner,991.34,5
Run CMC,950.34,5
Commish-ion Impossible,929.76,5
ACE of L.A.D.,937.72,4
Rod the Great's Team,894.06,4


In [14]:
scores = pd.read_csv('scores' + str(wk_count - 1) + '.csv')
scores = scores.set_index('Team')

scores

,totals,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Team,,,,,,,,,,,,,,,,,,
Commish-ion Impossible,0,75.92,93.38,117.16,135.20,99.02,117.18,111.64,93.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#CorruptCommish,0,103.02,84.60,84.66,138.50,95.70,82.70,127.24,62.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Abe W Memorial Champions,0,101.22,76.68,126.60,99.12,111.36,154.86,121.10,118.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACE of L.A.D.,0,124.08,122.00,91.12,152.18,115.46,82.64,83.42,52.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chip Kelly,0,111.36,128.32,90.52,122.56,139.52,113.00,102.72,154.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Takoma Queries (ex-Abe),0,88.28,87.54,103.44,91.30,70.20,82.82,124.94,124.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Naberhood Watch,0,89.82,103.08,151.02,94.38,121.02,92.38,98.38,48.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Run CMC,0,113.38,92.72,114.46,94.16,106.78,95.70,139.66,83.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Maseratis Not Jerseys,0,73.62,110.38,105.96,96.22,144.76,63.08,98.86,119.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
last_wk = pd.read_csv('weekly_standings' + str(wk_count - 1) + '.csv')
curr_wk = pd.read_csv('weekly_standings' + str(wk_count) + '.csv')

comp = curr_wk.merge(last_wk[['Team', 'PF']], on='Team')

comp['score'] = comp.PF_x - comp.PF_y

comp

,Team,Rank,W,L,T,PF_x,PA,PF_y,score
0,The Abe W Memorial Champions,1,7,2,0,1021.02,971.84,909.72,111.30
1,The Virtue Extollers,2,7,2,0,962.96,895.98,852.10,110.86
2,The Dream Team,3,6,3,0,974.62,852.16,884.66,89.96
3,Chip Kelly,4,5,4,0,1057.54,957.10,962.22,95.32
4,Kurtosis Warner,5,5,4,0,991.34,1022.56,832.52,158.82
5,Run CMC,6,5,4,0,950.34,984.52,840.82,109.52
6,Commish-ion Impossible,7,5,4,0,929.76,893.06,843.16,86.60
7,ACE of L.A.D.,8,4,5,0,937.72,949.84,823.78,113.94
8,Rod the Great's Team,9,4,5,0,894.06,941.58,799.36,94.70
9,Naberhood Watch,10,3,6,0,943.20,910.32,798.48,144.72


In [16]:
for team in scores.index:
    score = comp[comp.Team == team]['score']
    scores.at[team, str(wk_count)] = score

C:\Users\zachary.gozlan\AppData\Local\Temp\ipykernel_13040\1443111504.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  scores.at[team, str(wk_count)] = score


In [17]:
#scores = pd.read_csv('scores8.csv', index_col='Team')
scores

,totals,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Team,,,,,,,,,,,,,,,,,,
Commish-ion Impossible,0,75.92,93.38,117.16,135.20,99.02,117.18,111.64,93.66,86.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#CorruptCommish,0,103.02,84.60,84.66,138.50,95.70,82.70,127.24,62.60,119.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Abe W Memorial Champions,0,101.22,76.68,126.60,99.12,111.36,154.86,121.10,118.78,111.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACE of L.A.D.,0,124.08,122.00,91.12,152.18,115.46,82.64,83.42,52.88,113.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chip Kelly,0,111.36,128.32,90.52,122.56,139.52,113.00,102.72,154.22,95.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Takoma Queries (ex-Abe),0,88.28,87.54,103.44,91.30,70.20,82.82,124.94,124.06,100.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Naberhood Watch,0,89.82,103.08,151.02,94.38,121.02,92.38,98.38,48.40,144.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Run CMC,0,113.38,92.72,114.46,94.16,106.78,95.70,139.66,83.96,109.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Maseratis Not Jerseys,0,73.62,110.38,105.96,96.22,144.76,63.08,98.86,119.32,50.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
scores.columns = ['totals', 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]

scores.columns

Index(['totals', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='object')

In [19]:
scores.totals = scores[list(scores.columns)].sum(axis=1)

scores

,totals,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Team,,,,,,,,,,,,,,,,,,
Commish-ion Impossible,929.76,75.92,93.38,117.16,135.20,99.02,117.18,111.64,93.66,86.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#CorruptCommish,898.76,103.02,84.60,84.66,138.50,95.70,82.70,127.24,62.60,119.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Abe W Memorial Champions,1021.02,101.22,76.68,126.60,99.12,111.36,154.86,121.10,118.78,111.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACE of L.A.D.,937.72,124.08,122.00,91.12,152.18,115.46,82.64,83.42,52.88,113.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chip Kelly,1057.54,111.36,128.32,90.52,122.56,139.52,113.00,102.72,154.22,95.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Takoma Queries (ex-Abe),873.34,88.28,87.54,103.44,91.30,70.20,82.82,124.94,124.06,100.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Naberhood Watch,943.20,89.82,103.08,151.02,94.38,121.02,92.38,98.38,48.40,144.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Run CMC,950.34,113.38,92.72,114.46,94.16,106.78,95.70,139.66,83.96,109.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Maseratis Not Jerseys,862.96,73.62,110.38,105.96,96.22,144.76,63.08,98.86,119.32,50.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#current_oppPoints['W'] = [int(i.split("-")[0]) for i in current_oppPoints['W-L-T']] #fixes win issue
seed_table['W'] = current_oppPoints['W']

seed_table #current win count and point total

C:\Users\zachary.gozlan\AppData\Local\Temp\ipykernel_13040\449418957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seed_table['W'] = current_oppPoints['W']


,Points,W
Team,,
The Abe W Memorial Champions,1021.02,7
The Virtue Extollers,962.96,7
The Dream Team,974.62,6
Chip Kelly,1057.54,5
Kurtosis Warner,991.34,5
Run CMC,950.34,5
Commish-ion Impossible,929.76,5
ACE of L.A.D.,937.72,4
Rod the Great's Team,894.06,4


In [21]:
for i in range (wk_count+1, 18): #17 week season
    scores[i] = np.nan
    
scores

,totals,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
Team,,,,,,,,,,,,,,,,,,
Commish-ion Impossible,929.76,75.92,93.38,117.16,135.20,99.02,117.18,111.64,93.66,86.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#CorruptCommish,898.76,103.02,84.60,84.66,138.50,95.70,82.70,127.24,62.60,119.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Abe W Memorial Champions,1021.02,101.22,76.68,126.60,99.12,111.36,154.86,121.10,118.78,111.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACE of L.A.D.,937.72,124.08,122.00,91.12,152.18,115.46,82.64,83.42,52.88,113.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chip Kelly,1057.54,111.36,128.32,90.52,122.56,139.52,113.00,102.72,154.22,95.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Takoma Queries (ex-Abe),873.34,88.28,87.54,103.44,91.30,70.20,82.82,124.94,124.06,100.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Naberhood Watch,943.20,89.82,103.08,151.02,94.38,121.02,92.38,98.38,48.40,144.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Run CMC,950.34,113.38,92.72,114.46,94.16,106.78,95.70,139.66,83.96,109.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Maseratis Not Jerseys,862.96,73.62,110.38,105.96,96.22,144.76,63.08,98.86,119.32,50.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
wk_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

21

In [29]:
#using 2025 nfl season byes
bye_multiplier = [1, 1, 1, 1, 1, \
                 1.143, 1.067, 1.067, 1.231, \
                 1.143, 1.143, 1.067, 1.143, \
                 1, 1.143, 1, 1, 1, 1, \
                 1, 1, 1, 1] # additional weeks that don't exist to circumvent table runout error 

i = 1

for team in scores.index:
    print(team)
    final_scores = list(scores.loc[team][1:wk_count+1])
    
    try:
        neg_one = final_scores[-1] * bye_multiplier[wk_count-1]
    except IndexError:
        if wk_count >= 1:
            neg_one = avg * bye_multiplier[wk_count-1]
        else:
            neg_one = avg
        
    try:
        neg_two = final_scores[-2] * bye_multiplier[wk_count-2]
    except IndexError:
        if wk_count >= 2:
            neg_two = avg * bye_multiplier[wk_count-2]
        else:
            neg_two = avg
        
    try:
        neg_three = final_scores[-3] * bye_multiplier[wk_count-3]
    except IndexError:
        if wk_count >= 3:
            neg_three = avg * bye_multiplier[wk_count-3]
        else:
            neg_three = avg
        
    try:
        neg_four = final_scores[-4] * bye_multiplier[wk_count-4]
    except IndexError:
        if wk_count >= 4:
            neg_four = avg * bye_multiplier[wk_count-4]
        else:
            neg_four = avg
        
    try:
        neg_five = final_scores[-5] * bye_multiplier[wk_count-5]
    except IndexError:
        if wk_count >= 5:
            neg_five = avg * bye_multiplier[wk_count-5]
        else:
            neg_five = avg
    
    plus_one = ((neg_one+neg_two+neg_three+neg_four+neg_five)/5) * (1/bye_multiplier[wk_count])
    plus_two = ((neg_one+neg_two+neg_three+neg_four+avg)/5) * (1/bye_multiplier[wk_count+1])
    plus_three = ((neg_one+neg_two+neg_three+avg+avg)/5) * (1/bye_multiplier[wk_count+2])
    plus_four = ((neg_one+neg_two+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+3])
    plus_five = ((neg_one+avg+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+4]) #ONLY COMMENTED OUT LAST WEEKS
        
    for item in [plus_one, plus_two, plus_three, plus_four, plus_five]: # , plus_five]:
        final_scores.append(item)

    
    '''except IndexError:
        try:
            plus_one = ((neg_one+neg_two+neg_three+neg_four+neg_five)/5) * (1/bye_multiplier[wk_count])
            plus_two = ((neg_one+neg_two+neg_three+neg_four+avg)/5) * (1/bye_multiplier[wk_count+1])
            plus_three = ((neg_one+neg_two+neg_three+avg+avg)/5) * (1/bye_multiplier[wk_count+2])
            plus_four = ((neg_one+neg_two+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+3])

            for item in [plus_one, plus_two, plus_three, plus_four]:
                final_scores.append(item)
        

        except IndexError:
            plus_one = ((neg_one+neg_two+neg_three+neg_four+neg_five)/5) * (1/bye_multiplier[wk_count])
            plus_two = ((neg_one+neg_two+neg_three+neg_four+avg)/5) * (1/bye_multiplier[wk_count+1])
            plus_three = ((neg_one+neg_two+neg_three+avg+avg)/5) * (1/bye_multiplier[wk_count+2])'''

    i = wk_count
                
    '''for item in [plus_one, plus_two, plus_three]:
        final_scores.append(item)'''
    print(final_scores)
     
    while len(final_scores) < len(wk_list):
        final_scores.append(round(avg * (1/bye_multiplier[i+5]), 3))
        i = i + 1
        
    
    final_scores = [round(i, 2) for i in final_scores]
    scores.loc[team, wk_list] = final_scores

Commish-ion Impossible
[75.92, 93.38, 117.16, 135.2, 99.02, 117.18, 111.64, 93.66, 86.6, 97.74565879265091, 99.88564129483815, 102.74783505154637, 94.53890113735783, 110.32092]
#CorruptCommish
[103.02, 84.6, 84.66, 138.5, 95.7, 82.7, 127.24, 62.6, 119.74, 94.52061592300961, 97.24152580927384, 107.30257169634488, 95.878239720035, 118.479988]
The Abe W Memorial Champions
[101.22, 76.68, 126.6, 99.12, 111.36, 154.86, 121.1, 118.78, 111.3, 119.21736482939635, 119.19811723534559, 115.36312277413309, 104.54917935258094, 116.40206]
ACE of L.A.D.
[124.08, 122.0, 91.12, 152.18, 115.46, 82.64, 83.42, 52.88, 113.94, 86.72086789151356, 85.9842099737533, 95.25627741330835, 92.81419072615923, 117.05202799999999]
Chip Kelly
[111.36, 128.32, 90.52, 122.56, 139.52, 113.0, 102.72, 154.22, 95.32, 115.51581802274717, 110.56918635170605, 115.08789128397375, 107.72382502187227, 112.46778400000001]
Takoma Queries (ex-Abe)
[88.28, 87.54, 103.44, 91.3, 70.2, 82.82, 124.94, 124.06, 100.76, 97.03968853893264, 10

In [30]:
#scores.rename(index={"Bmores Finest":"Bmore's Finest"}, inplace=True)
scores.index.names = ['Team']

scores

,totals,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
Team,,,,,,,,,,,,,,,,,,,,,,
Commish-ion Impossible,929.76,75.92,93.38,117.16,135.20,99.02,117.18,111.64,93.66,86.60,97.75,99.89,102.75,94.54,110.32,97.33,111.25,111.25,111.25,111.25,111.25,111.25
#CorruptCommish,898.76,103.02,84.60,84.66,138.50,95.70,82.70,127.24,62.60,119.74,94.52,97.24,107.30,95.88,118.48,97.33,111.25,111.25,111.25,111.25,111.25,111.25
The Abe W Memorial Champions,1021.02,101.22,76.68,126.60,99.12,111.36,154.86,121.10,118.78,111.30,119.22,119.20,115.36,104.55,116.40,97.33,111.25,111.25,111.25,111.25,111.25,111.25
ACE of L.A.D.,937.72,124.08,122.00,91.12,152.18,115.46,82.64,83.42,52.88,113.94,86.72,85.98,95.26,92.81,117.05,97.33,111.25,111.25,111.25,111.25,111.25,111.25
Chip Kelly,1057.54,111.36,128.32,90.52,122.56,139.52,113.00,102.72,154.22,95.32,115.52,110.57,115.09,107.72,112.47,97.33,111.25,111.25,111.25,111.25,111.25,111.25
Takoma Queries (ex-Abe),873.34,88.28,87.54,103.44,91.30,70.20,82.82,124.94,124.06,100.76,97.04,104.22,114.76,103.26,113.81,97.33,111.25,111.25,111.25,111.25,111.25,111.25
Naberhood Watch,943.20,89.82,103.08,151.02,94.38,121.02,92.38,98.38,48.40,144.72,98.23,96.52,104.45,98.61,124.63,97.33,111.25,111.25,111.25,111.25,111.25,111.25
Run CMC,950.34,113.38,92.72,114.46,94.16,106.78,95.70,139.66,83.96,109.52,103.16,103.95,111.70,97.66,115.96,97.33,111.25,111.25,111.25,111.25,111.25,111.25
Maseratis Not Jerseys,862.96,73.62,110.38,105.96,96.22,144.76,63.08,98.86,119.32,50.76,89.61,83.75,97.05,91.61,101.50,97.33,111.25,111.25,111.25,111.25,111.25,111.25


In [31]:
seed_table
remaining_table = scores.drop(['totals'], axis=1)

#remaining_table

In [35]:
#displays current week's head to head win probabilities

remaining_sched = pd.read_csv('OneUSAC2025_sched.csv') #generated from schedule thing above
remaining_sched = remaining_sched[remaining_sched['Week'] > wk_count]
remaining_sched.head(12)

remaining_sched['TeamScore'] = [fc.score_generator2(remaining_table, j.Team, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['OppoScore'] = [fc.score_generator2(remaining_table, j.Opponent, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in remaining_sched.iterrows()] #there won't be ties
remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
remaining_sched['prob'] = [fc.prob_10k(j.TeamScore, j.OppoScore, wobble) for i,j in remaining_sched.iterrows()]

remaining_sched.head(7)

,Week,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win,prob
63,10,The Virtue Extollers,Run CMC,102.76,103.16,0,1,0.49531
64,10,Commish-ion Impossible,The Terrible Towels,97.75,91.89,1,0,0.56341
65,10,#CorruptCommish,Rod the Great's Team,94.52,95.92,0,1,0.48437
66,10,The Abe W Memorial Champions,Maseratis Not Jerseys,119.22,89.61,1,0,0.78782
67,10,ACE of L.A.D.,Takoma Queries (ex-Abe),86.72,97.04,0,1,0.39048
68,10,Chip Kelly,Naberhood Watch,115.52,98.23,1,0,0.68056
69,10,The Dream Team,Kurtosis Warner,99.59,120.24,0,1,0.28809


## One Loop

The below cells run through one loop of the final output; the season is played out based on the probabilities generated above and the remaining schedule.

In [36]:
remaining_sched = pd.read_csv('OneUSAC2025_sched.csv') #generated from schedule thing above
remaining_sched = remaining_sched[remaining_sched['Week'] > wk_count]
remaining_sched.head(14)

,Week,Team,Opponent
63,10,The Virtue Extollers,Run CMC
64,10,Commish-ion Impossible,The Terrible Towels
65,10,#CorruptCommish,Rod the Great's Team
66,10,The Abe W Memorial Champions,Maseratis Not Jerseys
67,10,ACE of L.A.D.,Takoma Queries (ex-Abe)
68,10,Chip Kelly,Naberhood Watch
69,10,The Dream Team,Kurtosis Warner
70,11,The Virtue Extollers,ACE of L.A.D.
71,11,Commish-ion Impossible,Maseratis Not Jerseys
72,11,#CorruptCommish,The Abe W Memorial Champions


In [37]:
remaining_sched['TeamScore'] = [fc.score_generator(remaining_table, j.Team, j.Week, wobble) for i,j in remaining_sched.iterrows()]
remaining_sched['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, j.Week, wobble) for i,j in remaining_sched.iterrows()]
remaining_sched['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in remaining_sched.iterrows()] #there won't be ties
remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
remaining_sched.head(7)

,Week,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win
63,10,The Virtue Extollers,Run CMC,130.140543,60.689179,1,0
64,10,Commish-ion Impossible,The Terrible Towels,76.215393,87.116410,0,1
65,10,#CorruptCommish,Rod the Great's Team,112.215554,77.525054,1,0
66,10,The Abe W Memorial Champions,Maseratis Not Jerseys,89.234014,95.249679,0,1
67,10,ACE of L.A.D.,Takoma Queries (ex-Abe),129.233521,113.003980,1,0
68,10,Chip Kelly,Naberhood Watch,124.867632,103.324348,1,0
69,10,The Dream Team,Kurtosis Warner,101.051478,165.859372,0,1


In [38]:
remaining_sched_1 = remaining_sched.copy()
remaining_sched_2 = remaining_sched.copy()

remaining_sched_2['Team'] = remaining_sched['Opponent']
remaining_sched_2['Opponent'] = remaining_sched['Team']

remaining_sched_2['TeamScore'] = remaining_sched['OppoScore']
remaining_sched_2['OppoScore'] = remaining_sched['TeamScore']

remaining_sched_EV = pd.concat([remaining_sched_1,remaining_sched_2], ignore_index=True)
remaining_sched_EV = remaining_sched_EV.reset_index()
remaining_sched_EV = remaining_sched_EV[['Week', 'Team', 'TeamScore', 'OppoScore']]

weekly_leaders = list(remaining_sched_EV.loc[list(remaining_sched_EV.groupby('Week')['TeamScore'].idxmax())]['Team'])

remaining_sched_EV.loc[list(remaining_sched_EV.groupby('Week')['TeamScore'].idxmax())]  #new 2023

,Week,Team,TeamScore,OppoScore
41,10,Kurtosis Warner,165.859372,101.051478
7,11,The Virtue Extollers,150.740195,57.913335
53,12,Takoma Queries (ex-Abe),140.520423,111.738916
21,13,The Virtue Extollers,139.889898,63.173095
32,14,ACE of L.A.D.,176.996813,129.723757


In [39]:
defense = pd.DataFrame(remaining_sched_EV.groupby('Team')['OppoScore'].sum())

defense

,OppoScore
Team,
#CorruptCommish,519.411627
ACE of L.A.D.,566.854553
Chip Kelly,578.493964
Commish-ion Impossible,551.083921
Kurtosis Warner,645.216675
Maseratis Not Jerseys,463.533277
Naberhood Watch,470.007648
Rod the Great's Team,505.298953
Run CMC,512.617467


In [40]:
#simplify that frame down to the number of wins each team gains in that simmed run of the league

wins1 = remaining_sched[['Team', 'team_win', 'TeamScore']]
wins2 = remaining_sched[['Opponent', 'oppo_win', 'OppoScore']]
wins1 = wins1.groupby('Team').sum()
wins2 = wins2.groupby('Opponent').sum()
wins2 = wins2.rename(columns={'oppo_win':'team_win'})

wins3 = pd.concat([wins1, wins2])
wins3['Team'] = wins3.index
simmed_wins = wins3.groupby('Team').sum()
simmed_wins['Points'] = simmed_wins.OppoScore + simmed_wins.TeamScore
simmed_wins = simmed_wins.drop(['OppoScore', 'TeamScore'], axis=1)

simmed_wins

,team_win,Points
Team,,
#CorruptCommish,2,524.556160
ACE of L.A.D.,3,495.880703
Chip Kelly,2,481.800505
Commish-ion Impossible,2,540.577792
Kurtosis Warner,3,668.393113
Maseratis Not Jerseys,4,572.404412
Naberhood Watch,4,511.191155
Rod the Great's Team,1,437.901389
Run CMC,2,471.179954


In [41]:
#read in the current standings frame

current_standings = seed_table
current_standings = current_standings[['Points', 'W']]
new_standings = current_standings

new_standings

,Points,W
Team,,
The Abe W Memorial Champions,1021.02,7
The Virtue Extollers,962.96,7
The Dream Team,974.62,6
Chip Kelly,1057.54,5
Kurtosis Warner,991.34,5
Run CMC,950.34,5
Commish-ion Impossible,929.76,5
ACE of L.A.D.,937.72,4
Rod the Great's Team,894.06,4


In [42]:
#read in the current standings frame

current_standings = seed_table.copy()

current_standings

for team in scores.index:
    new_standings.at[team, 'W'] = current_standings.at[team, 'W'] + simmed_wins.at[team, 'team_win']
    new_standings.at[team, 'Points'] = current_standings.at[team, 'Points'] + simmed_wins.at[team, 'Points']
    
new_standings


,Points,W
Team,,
The Abe W Memorial Champions,1566.299961,10
The Virtue Extollers,1648.359591,11
The Dream Team,1458.429328,7
Chip Kelly,1539.340505,7
Kurtosis Warner,1659.733113,8
Run CMC,1421.519954,7
Commish-ion Impossible,1470.337792,7
ACE of L.A.D.,1433.600703,7
Rod the Great's Team,1331.961389,5


In [43]:


for_points = new_standings.sort_values(by=['Points'], ascending=False) #new 2023
for_points.index[0]

'Kurtosis Warner'

In [44]:
new_standings

,Points,W
Team,,
The Abe W Memorial Champions,1566.299961,10
The Virtue Extollers,1648.359591,11
The Dream Team,1458.429328,7
Chip Kelly,1539.340505,7
Kurtosis Warner,1659.733113,8
Run CMC,1421.519954,7
Commish-ion Impossible,1470.337792,7
ACE of L.A.D.,1433.600703,7
Rod the Great's Team,1331.961389,5


In [45]:
#rearrange the simulated outcome frame by wins and then points (the tiebreaker metric) to determine the "place" of each team
#in the simmed league

simmed_standings = new_standings.sort_values(by=['W', 'Points'], ascending=False)
simmed_standings['Points'] = current_standings['Points'] + simmed_wins['Points'] #final week only don't add in
simmed_standings = simmed_standings.sort_values(by=['W', 'Points'], ascending=False)
simmed_standings['place'] = np.arange(1,15)

simmed_standings

,Points,W,place
Team,,,
The Virtue Extollers,1648.359591,11,1
The Abe W Memorial Champions,1566.299961,10,2
Kurtosis Warner,1659.733113,8,3
Chip Kelly,1539.340505,7,4
Commish-ion Impossible,1470.337792,7,5
The Dream Team,1458.429328,7,6
Naberhood Watch,1454.391155,7,7
Maseratis Not Jerseys,1435.364412,7,8
ACE of L.A.D.,1433.600703,7,9


In [46]:
simmed_standings[['Points']].sort_values('Points', ascending=False).index[0]

simmed_standings

,Points,W,place
Team,,,
The Virtue Extollers,1648.359591,11,1
The Abe W Memorial Champions,1566.299961,10,2
Kurtosis Warner,1659.733113,8,3
Chip Kelly,1539.340505,7,4
Commish-ion Impossible,1470.337792,7,5
The Dream Team,1458.429328,7,6
Naberhood Watch,1454.391155,7,7
Maseratis Not Jerseys,1435.364412,7,8
ACE of L.A.D.,1433.600703,7,9


In [47]:
simmed_standings['Team'] = simmed_standings.index

simmed_standings = simmed_standings[['Team', 'Points', 'W', 'place']]

x = pd.DataFrame.to_numpy(simmed_standings)

y = pd.DataFrame(x, columns=['Team', 'Points', 'W', 'place'])
y = y.set_index('Team')

y

,Points,W,place
Team,,,
The Virtue Extollers,1648.359591,11,1
The Abe W Memorial Champions,1566.299961,10,2
Kurtosis Warner,1659.733113,8,3
Chip Kelly,1539.340505,7,4
Commish-ion Impossible,1470.337792,7,5
The Dream Team,1458.429328,7,6
Naberhood Watch,1454.391155,7,7
Maseratis Not Jerseys,1435.364412,7,8
ACE of L.A.D.,1433.600703,7,9


In [48]:
defense_final = pd.merge(current_oppPoints, defense, left_index=True, right_index=True)

defense_final['totalOpp'] = defense_final['PA'] + defense_final['OppoScore']

print(defense_final['totalOpp'].sort_values(ascending=False))

defense_final[['OppoScore']].sort_values('OppoScore', ascending=False).index[0] #team defense loser

Team
Kurtosis Warner                 1667.776675
Takoma Queries (ex-Abe)         1574.447863
Chip Kelly                      1535.593964
The Abe W Memorial Champions    1518.185260
ACE of L.A.D.                   1516.694553
Run CMC                         1497.137467
Maseratis Not Jerseys           1471.033277
Rod the Great's Team            1446.878953
Commish-ion Impossible          1444.143921
The Dream Team                  1432.360114
The Terrible Towels             1417.728704
#CorruptCommish                 1391.171627
Naberhood Watch                 1380.327648
The Virtue Extollers            1344.527917
Name: totalOpp, dtype: float64


'Kurtosis Warner'

In [49]:
team_list = []
points_list = []
w_list = []
place_list = []

In [50]:
team_list.extend(y.index)
points_list.extend(y.Points)
w_list.extend(y.W)
place_list.extend(y.place)

In [51]:
array = pd.DataFrame()
array['team'] = team_list
array['Points'] = points_list
array['W'] = w_list
array['place'] = place_list

array

,team,Points,W,place
0,The Virtue Extollers,1648.359591,11,1
1,The Abe W Memorial Champions,1566.299961,10,2
2,Kurtosis Warner,1659.733113,8,3
3,Chip Kelly,1539.340505,7,4
4,Commish-ion Impossible,1470.337792,7,5
5,The Dream Team,1458.429328,7,6
6,Naberhood Watch,1454.391155,7,7
7,Maseratis Not Jerseys,1435.364412,7,8
8,ACE of L.A.D.,1433.600703,7,9
9,Run CMC,1421.519954,7,10


In [52]:
dict_test = simmed_standings[['place']].to_dict()
dict2 = dict_test.pop('place')
inv_map = {v: k for k, v in dict2.items()}
inv_map

inv_map.update( {15 : 'bye'} )

print('Seeds')
print(inv_map)
print('\n')

matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[15], inv_map[5], inv_map[6], inv_map[15], inv_map[15],inv_map[11], inv_map[12], inv_map[15]]
        }

playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])

playoff_r1

Seeds
{1: 'The Virtue Extollers', 2: 'The Abe W Memorial Champions', 3: 'Kurtosis Warner', 4: 'Chip Kelly', 5: 'Commish-ion Impossible', 6: 'The Dream Team', 7: 'Naberhood Watch', 8: 'Maseratis Not Jerseys', 9: 'ACE of L.A.D.', 10: 'Run CMC', 11: 'Takoma Queries (ex-Abe)', 12: '#CorruptCommish', 13: "Rod the Great's Team", 14: 'The Terrible Towels', 15: 'bye'}




,Team,Opponent
0,The Virtue Extollers,bye
1,Chip Kelly,Commish-ion Impossible
2,Kurtosis Warner,The Dream Team
3,The Abe W Memorial Champions,bye
4,Naberhood Watch,bye
5,Run CMC,Takoma Queries (ex-Abe)
6,ACE of L.A.D.,#CorruptCommish
7,Maseratis Not Jerseys,bye


In [54]:
dict_test = simmed_standings[['place']].to_dict()
dict2 = dict_test.pop('place')
inv_map = {v: k for k, v in dict2.items()}
inv_map

inv_map.update( {playoffs_start_wk : 'bye'} )

print('Seeds')
print(inv_map)
print('\n')

matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[15], inv_map[5], inv_map[6], inv_map[15], inv_map[15],inv_map[11], inv_map[12], inv_map[15]]
        }

playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])

print('Round 1')

playoff_r1['TeamScore'] = [fc.score_generator(remaining_table, j.Team, playoffs_start_wk, wobble) for i,j in playoff_r1.iterrows()]
playoff_r1['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, playoffs_start_wk, wobble) for i,j in playoff_r1.iterrows()]
playoff_r1['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r1.iterrows()] #there won't be ties
playoff_r1['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r1.iterrows()]

winners_r1 = []
losers_r1 = []
for index, row in playoff_r1.iterrows():
    if row['team_win'] == 1:
        winners_r1.append(row['Team'])
    else:
        winners_r1.append(row['Opponent'])
        
print(winners_r1)
print('\n')

matchups_r2 = {'Team': [winners_r1[0], winners_r1[2], winners_r1[4], winners_r1[6]], 
        'Opponent': [winners_r1[1], winners_r1[3], winners_r1[5], winners_r1[7]]
        }

playoff_r2 = pd.DataFrame(matchups_r2, columns = ['Team', 'Opponent'])

playoff_r2['TeamScore'] = [fc.score_generator(remaining_table, j.Team, playoffs_start_wk + 1, wobble) for i,j in playoff_r2.iterrows()]
playoff_r2['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, playoffs_start_wk + 1, wobble) for i,j in playoff_r2.iterrows()]
playoff_r2['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r2.iterrows()] #there won't be ties
playoff_r2['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r2.iterrows()]

winners_r2 = []
losers_r2 = []
for index, row in playoff_r2.iterrows():
    if row['team_win'] == 1:
        winners_r2.append(row['Team'])
        losers_r2.append(row['Opponent'])
    else:
        winners_r2.append(row['Opponent'])
        losers_r2.append(row['Team'])

print('Round 2')
print(winners_r2)
print('\n')

matchups_r3 = {'Team': [winners_r2[0], losers_r2[0], winners_r2[2]], 
        'Opponent': [winners_r2[1], losers_r2[1], winners_r2[3]]
        }

playoff_r3 = pd.DataFrame(matchups_r3, columns = ['Team', 'Opponent'])

playoff_r3['TeamScore'] = [fc.score_generator(remaining_table, j.Team, playoffs_start_wk + 2, wobble) for i,j in playoff_r3.iterrows()]
playoff_r3['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, playoffs_start_wk + 2, wobble) for i,j in playoff_r3.iterrows()]
playoff_r3['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r3.iterrows()] #there won't be ties
playoff_r3['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r3.iterrows()]

winners_r3 = []
losers_r3 = []
for index, row in playoff_r3.iterrows():
    if row['team_win'] == 1:
        winners_r3.append(row['Team'])
        losers_r3.append(row['Opponent'])
    else:
        winners_r3.append(row['Opponent'])
        losers_r3.append(row['Team'])
        
print('Round 3')
print(winners_r3)
print('\n')

Seeds
{1: 'The Virtue Extollers', 2: 'The Abe W Memorial Champions', 3: 'Kurtosis Warner', 4: 'Chip Kelly', 5: 'Commish-ion Impossible', 6: 'The Dream Team', 7: 'Naberhood Watch', 8: 'Maseratis Not Jerseys', 9: 'ACE of L.A.D.', 10: 'Run CMC', 11: 'Takoma Queries (ex-Abe)', 12: '#CorruptCommish', 13: "Rod the Great's Team", 14: 'The Terrible Towels', 15: 'bye'}


Round 1
['The Virtue Extollers', 'Chip Kelly', 'The Dream Team', 'The Abe W Memorial Champions', 'Naberhood Watch', 'Takoma Queries (ex-Abe)', 'ACE of L.A.D.', 'Maseratis Not Jerseys']


Round 2
['The Virtue Extollers', 'The Abe W Memorial Champions', 'Naberhood Watch', 'Maseratis Not Jerseys']


Round 3
['The Virtue Extollers', 'The Dream Team', 'Naberhood Watch']




In [55]:
champ_table = pd.DataFrame(columns=['First', 'Second', 'Third', 'Consolation'])

matchups = []
for index, row in champ_table.iterrows():
    championship = []
    championship.append(row['First'])
    championship.append(row['Second'])
    championship = sorted(championship)
    matchups.append(str(championship))
    
counter=collections.Counter(matchups)
counter

newrow = {'First' : winners_r3[0], 'Second' : losers_r3[0], 'Third' : winners_r3[1], 'Consolation' : winners_r3[2] } 
champ_table = pd.concat([champ_table, pd.DataFrame([newrow])],ignore_index=True)

champ_table

,First,Second,Third,Consolation
0,The Virtue Extollers,The Abe W Memorial Champions,The Dream Team,Naberhood Watch


In [56]:
simmed_wins

,team_win,Points
Team,,
#CorruptCommish,2,524.556160
ACE of L.A.D.,3,495.880703
Chip Kelly,2,481.800505
Commish-ion Impossible,2,540.577792
Kurtosis Warner,3,668.393113
Maseratis Not Jerseys,4,572.404412
Naberhood Watch,4,511.191155
Rod the Great's Team,1,437.901389
Run CMC,2,471.179954


## Does The Deal

Runs the above loop *number_sims* times, usually 10,000 (~20 minute runtime).

In [57]:
#this is the equivalent of looping the previous five code blocks n number of times to get a distribution of possible outcomes
#for each team

number_sims = 10000 #one hour for full schedule at 10k sims
#all_predictions = pd.DataFrame(columns=['Points', 'W', 'place'])
i = 0

team_list = []
points_list = []
w_list = []
place_list = []


first_list = []
second_list = []
third_list = []
consolation_list = []

week_winners = []
defense_losers = []
scoring_leaders = []

start = time.time()

while i < number_sims: 
    
    # game sims #
    
    current_standings = seed_table.copy()
    
    remaining_table = scores.drop(['totals'], axis=1) #automate this
    
    remaining_sched['TeamScore'] = [fc.score_generator(remaining_table, j.Team, j.Week, wobble) for i,j in remaining_sched.iterrows()]
    remaining_sched['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, j.Week, wobble) for i,j in remaining_sched.iterrows()]
    remaining_sched['team_win'] = [int(j.TeamScore>j.OppoScore) for i,j in remaining_sched.iterrows()]
    remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
     
    # find week-by-week winners #
    
    remaining_sched_1 = remaining_sched.copy()
    remaining_sched_2 = remaining_sched.copy()

    remaining_sched_2['Team'] = remaining_sched['Opponent']
    remaining_sched_2['Opponent'] = remaining_sched['Team']

    remaining_sched_2['TeamScore'] = remaining_sched['OppoScore']
    remaining_sched_2['OppoScore'] = remaining_sched['TeamScore']

    remaining_sched_EV = pd.concat([remaining_sched_1,remaining_sched_2], ignore_index=True)
    remaining_sched_EV = remaining_sched_EV.reset_index()
    remaining_sched_EV = remaining_sched_EV[['Week', 'Team', 'TeamScore', 'OppoScore']]

    weekly_leaders = list(remaining_sched_EV.loc[list(remaining_sched_EV.groupby('Week')['TeamScore'].idxmax())]['Team'])
    
    week_winners.extend(weekly_leaders)

    
    # defense loser # 
    
    defense = pd.DataFrame(remaining_sched_EV.groupby('Team')['OppoScore'].sum())
    defense_final = pd.merge(current_oppPoints, defense, left_index=True, right_index=True)
    defense_final['totalOpp'] = defense_final['PA'] + defense_final['OppoScore']
    defense_loser = defense_final[['OppoScore']].sort_values('OppoScore', ascending=False).index[0] #team defense loser
        
    defense_losers.append(defense_loser)

    # standings #
    
    wins1 = remaining_sched[['Team', 'team_win', 'TeamScore']]
    wins2 = remaining_sched[['Opponent', 'oppo_win', 'OppoScore']]
    wins1 = wins1.groupby('Team').sum()
    wins2 = wins2.groupby('Opponent').sum()
    wins2 = wins2.rename(columns={'oppo_win':'team_win'})

    wins3 = pd.concat([wins1, wins2])
    wins3['Team'] = wins3.index
    simmed_wins = wins3.groupby('Team').sum()
    simmed_wins['Points'] = simmed_wins.OppoScore + simmed_wins.TeamScore
    simmed_wins = simmed_wins.drop(['OppoScore', 'TeamScore'], axis=1)
    
    
    ###FINAL WEEK: UNCOMMENT THIS CODE AND COMMENT OUT LINES BELOW
    ###new_standings = current_standings[['Points', 'W']]

    for team in team_list[:14]:
        new_standings.at[team, 'W'] = new_standings.at[team, 'W'] + simmed_wins.at[team, 'team_win']
    
     
    #rearrange the simulated outcome frame by wins and then points (the tiebreaker metric) to determine the "place" of each team
    #in the simmed league

    simmed_standings = current_standings.sort_values(by=['W', 'Points'], ascending=False)
    simmed_standings['W'] = current_standings['W'] + simmed_wins['team_win']
    simmed_standings['Points'] = current_standings['Points'] + simmed_wins['Points']
    simmed_standings = simmed_standings.sort_values(by=['W', 'Points'], ascending=False)
    simmed_standings['place'] = np.arange(1,15)
    
    scoring_leader = simmed_standings[['Points']].sort_values('Points', ascending=False).index[0]
    scoring_leaders.append(scoring_leader)
    
    #ranking assignment
    simmed_standings['place'] = np.arange(1,15)
    
    #all_predictions = pd.concat([all_predictions, simmed_standings])
    
    team_list.extend(simmed_standings.index)
    points_list.extend(simmed_standings.Points)
    w_list.extend(simmed_standings.W)
    place_list.extend(simmed_standings.place)

    
    ## PLAYOFF SIMS
    
    dict_test = simmed_standings[['place']].to_dict()
    dict2 = dict_test.pop('place')
    inv_map = {v: k for k, v in dict2.items()}
    inv_map

    inv_map.update( {playoffs_start_wk : 'bye'} )
    
    matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[15], inv_map[5], inv_map[6], inv_map[15], inv_map[15],inv_map[11], inv_map[12], inv_map[15]]
        }

    playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])

    playoff_r1['TeamScore'] = [fc.score_generator(remaining_table, j.Team, playoffs_start_wk, wobble) for i,j in playoff_r1.iterrows()]
    playoff_r1['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, playoffs_start_wk, wobble) for i,j in playoff_r1.iterrows()]
    playoff_r1['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r1.iterrows()] #there won't be ties
    playoff_r1['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r1.iterrows()]

    winners_r1 = []
    losers_r1 = []
    for index, row in playoff_r1.iterrows():
        if row['team_win'] == 1:
            winners_r1.append(row['Team'])
        else:
            winners_r1.append(row['Opponent'])

    matchups_r2 = {'Team': [winners_r1[0], winners_r1[2], winners_r1[4], winners_r1[6]], 
            'Opponent': [winners_r1[1], winners_r1[3], winners_r1[5], winners_r1[7]]
            }

    playoff_r2 = pd.DataFrame(matchups_r2, columns = ['Team', 'Opponent'])

    playoff_r2['TeamScore'] = [fc.score_generator(remaining_table, j.Team, playoffs_start_wk + 1, wobble) for i,j in playoff_r2.iterrows()]
    playoff_r2['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, playoffs_start_wk + 1, wobble) for i,j in playoff_r2.iterrows()]
    playoff_r2['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r2.iterrows()] #there won't be ties
    playoff_r2['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r2.iterrows()]

    winners_r2 = []
    losers_r2 = []
    for index, row in playoff_r2.iterrows():
        if row['team_win'] == 1:
            winners_r2.append(row['Team'])
            losers_r2.append(row['Opponent'])
        else:
            winners_r2.append(row['Opponent'])
            losers_r2.append(row['Team'])

    matchups_r3 = {'Team': [winners_r2[0], losers_r2[0], winners_r2[2]], 
            'Opponent': [winners_r2[1], losers_r2[1], winners_r2[3]]
            }

    playoff_r3 = pd.DataFrame(matchups_r3, columns = ['Team', 'Opponent'])

    playoff_r3['TeamScore'] = [fc.score_generator(remaining_table, j.Team, playoffs_start_wk + 2, wobble) for i,j in playoff_r3.iterrows()]
    playoff_r3['OppoScore'] = [fc.score_generator(remaining_table, j.Opponent, playoffs_start_wk + 2, wobble) for i,j in playoff_r3.iterrows()]
    playoff_r3['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r3.iterrows()] #there won't be ties
    playoff_r3['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r3.iterrows()]

    winners_r3 = []
    losers_r3 = []
    for index, row in playoff_r3.iterrows():
        if row['team_win'] == 1:
            winners_r3.append(row['Team'])
            losers_r3.append(row['Opponent'])
        else:
            winners_r3.append(row['Opponent'])
            losers_r3.append(row['Team'])
            
    matchups = []
    '''for index, row in champ_table.iterrows():
        championship = []
        championship.append(row['First'])
        championship.append(row['Second'])
        championship = sorted(championship)
        matchups.append(str(championship))'''

    counter=collections.Counter(matchups)
    counter

    first_list.append(winners_r3[0])
    second_list.append(losers_r3[0])
    third_list.append(winners_r3[1])
    consolation_list.append(winners_r3[2])


    i = i + 1
    if (i % 100 == 0):
        marker = time.time()
        print(str(i/100) + '%') #totally unnecessary, just gives a sense of progress
        print(marker - start)
        

1.0%
2.6558923721313477
2.0%
5.413599252700806
3.0%
8.399509191513062
4.0%
11.252409934997559
5.0%
13.826905012130737
6.0%
16.50732421875
7.0%
19.603272438049316
8.0%
22.102317810058594
9.0%
25.242868185043335
10.0%
28.311744689941406
11.0%
31.26003122329712
12.0%
34.278568983078
13.0%
37.36392641067505
14.0%
40.26300501823425
15.0%
43.38352632522583
16.0%
46.15160536766052
17.0%
48.93228626251221
18.0%
51.73660087585449
19.0%
54.55123972892761
20.0%
57.38932490348816
21.0%
60.05730581283569
22.0%
62.87572693824768
23.0%
66.03559947013855
24.0%
69.18756365776062
25.0%
72.37015986442566
26.0%
75.49946069717407
27.0%
78.9588270187378
28.0%
82.16840410232544
29.0%
85.41668009757996
30.0%
89.08989071846008
31.0%
92.26732969284058
32.0%
95.92314147949219
33.0%
99.70401048660278
34.0%
103.15637493133545
35.0%
106.78760862350464
36.0%
110.44041228294373
37.0%
113.9291353225708
38.0%
117.56650733947754
39.0%
120.99440145492554
40.0%
124.43210625648499
41.0%
128.12637877464294
42.0%
131.7646722

In [58]:
array = pd.DataFrame()
array['team'] = team_list
array['Points'] = points_list
array['W'] = w_list
array['place'] = place_list

all_predictions = array.set_index('team')

all_predictions.head(28)

,Points,W,place
team,,,
The Abe W Memorial Champions,1622.111472,11,1
The Virtue Extollers,1525.702493,10,2
Kurtosis Warner,1659.517566,9,3
The Dream Team,1504.607135,9,4
Chip Kelly,1666.707335,7,5
ACE of L.A.D.,1540.815053,7,6
Run CMC,1470.151831,7,7
Takoma Queries (ex-Abe),1351.791647,7,8
Naberhood Watch,1490.313840,6,9


In [59]:
week_win_frame = pd.DataFrame([[x,week_winners.count(x)] for x in set(seed_table.index)])
week_win_frame.columns = ['Team', 'Week Wins']
week_win_frame = week_win_frame.set_index('Team')
week_win_frame['Week Wins'] = week_win_frame['Week Wins'] / number_sims

week_win_frame

,Week Wins
Team,
Chip Kelly,0.5213
Takoma Queries (ex-Abe),0.3484
#CorruptCommish,0.2647
The Terrible Towels,0.2170
ACE of L.A.D.,0.1768
The Virtue Extollers,0.3604
The Dream Team,0.2798
Commish-ion Impossible,0.2297
Run CMC,0.3291


In [60]:
champ_table = pd.DataFrame()

champ_table['First'] = first_list
champ_table['Second'] = second_list
champ_table['Third'] = third_list
champ_table['Consolation'] = consolation_list
champ_table['Defense Loser'] = defense_losers
champ_table['Scoring Leader'] = scoring_leaders

champ_table

,First,Second,Third,Consolation,Defense Loser,Scoring Leader
0,The Virtue Extollers,Chip Kelly,The Abe W Memorial Champions,Naberhood Watch,Rod the Great's Team,Chip Kelly
1,The Abe W Memorial Champions,The Virtue Extollers,Kurtosis Warner,ACE of L.A.D.,ACE of L.A.D.,Kurtosis Warner
2,The Virtue Extollers,Kurtosis Warner,Run CMC,#CorruptCommish,ACE of L.A.D.,The Abe W Memorial Champions
3,The Abe W Memorial Champions,The Dream Team,Run CMC,The Virtue Extollers,#CorruptCommish,Chip Kelly
4,Rod the Great's Team,The Virtue Extollers,Kurtosis Warner,Run CMC,Chip Kelly,Chip Kelly
...,...,...,...,...,...,...
9995,Takoma Queries (ex-Abe),The Abe W Memorial Champions,Kurtosis Warner,The Dream Team,The Virtue Extollers,Kurtosis Warner
9996,Kurtosis Warner,The Abe W Memorial Champions,The Dream Team,ACE of L.A.D.,ACE of L.A.D.,Chip Kelly
9997,Chip Kelly,The Dream Team,The Abe W Memorial Champions,The Terrible Towels,The Virtue Extollers,Chip Kelly
9998,Chip Kelly,The Virtue Extollers,The Abe W Memorial Champions,Rod the Great's Team,The Terrible Towels,Chip Kelly


In [61]:
pd.DataFrame(champ_table['First'].value_counts())

,count
First,
The Abe W Memorial Champions,2268
The Virtue Extollers,1801
Kurtosis Warner,1526
Chip Kelly,1165
The Dream Team,1146
Run CMC,837
Commish-ion Impossible,690
Rod the Great's Team,183
ACE of L.A.D.,129


In [62]:
first_table = pd.DataFrame(champ_table['First'].value_counts())
second_table = pd.DataFrame(champ_table['Second'].value_counts())
third_table = pd.DataFrame(champ_table['Third'].value_counts())
consolation_table = pd.DataFrame(champ_table['Consolation'].value_counts())
defense_table = pd.DataFrame(champ_table['Defense Loser'].value_counts())
scoring_table = pd.DataFrame(champ_table['Scoring Leader'].value_counts())

dfs = [first_table, second_table, third_table, consolation_table, defense_table, scoring_table]

final = first_table.merge(second_table, how='outer', on=None, left_index=True, right_index=True, suffixes=['a1', 'a2'])
final = final.merge(third_table, how='outer', on=None, left_index=True, right_index=True, suffixes=['b1', 'b2'])
final = final.merge(consolation_table, how='outer', on=None, left_index=True, right_index=True, suffixes=['c1', 'c2'])
final = final.merge(defense_table, how='outer', on=None, left_index=True, right_index=True, suffixes=['d1', 'd2'])
final = final.merge(scoring_table, how='outer', on=None, left_index=True, right_index=True, suffixes=['e1', 'e2'])
final = final.merge(week_win_frame, how='outer', on=None, left_index=True, right_index=True, suffixes=['f1', 'f2'])

final.columns = ['First', 'Second', 'Third', 'Consolation', 'Defense Loser', 'Scoring Leader', 'Week Wins']

final = final.fillna(0).sort_values(['First', 'Second', 'Third', 'Consolation'], ascending=False)

final['Weeks Won'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]

final.at['Kurtosis Warner', 'Weeks Won'] = 2 #wk 1, 9
final.at['The Dream Team', 'Weeks Won'] = 1 #wk 2
final.at['Naberhood Watch', 'Weeks Won'] = 1 #wk 3
final.at['ACE of L.A.D.', 'Weeks Won'] = 1 #wk 4
final.at['Maseratis Not Jerseys', 'Weeks Won'] = 1 #wk 5
final.at['The Abe W Memorial Champions', 'Weeks Won'] = 1 #wk 6
final.at['Run CMC', 'Weeks Won'] = 1 #wk 7
final.at['Chip Kelly', 'Weeks Won'] = 1 #wk 8

#final.at['ACE of L.A.D.', 'Weeks Won'] = 3 #wk 1, 4,6
#final.at['CTE Pros (Abe)', 'Weeks Won'] = 5 #wk 2, 3, 9, 10, 12
#final.at['The Integrity Maintainers', 'Weeks Won'] = 2 #wk 5, 13
#final.at['5 and Juan', 'Weeks Won'] = 2 #wk 7, 11
#final.at["Rod the Great's Team", 'Weeks Won'] = 1 #wk 8

final['value'] = round((((final.First/number_sims)*100) + \
                        ((final.Second/number_sims)*50) + \
                        ((final.Third/number_sims)*25) + \
                        ((final['Scoring Leader']/number_sims)*25) + \
                        ((final['Week Wins'])*10) +\
                        ((final['Weeks Won'])*10) +\
                        ((final['Defense Loser']/number_sims)*10)),2)

final.columns = ['1st (Playoff)', '2nd (Playoff)', '3rd (Playoff)', 'Win Consolation', 'Overall Worst Defense',\
                'Season Scoring Leader', 'Remaining Week Top Scores (Proj.)', 'Past Week Top Scores', 'Total Team EV']

final.sort_values('Total Team EV', ascending=False)

,1st (Playoff),2nd (Playoff),3rd (Playoff),Win Consolation,Overall Worst Defense,Season Scoring Leader,Remaining Week Top Scores (Proj.),Past Week Top Scores,Total Team EV
Kurtosis Warner,1526,1507,1578,186,675,3337.0,1.0149,2,65.91
The Abe W Memorial Champions,2268,2346,2269,12,336,2389.0,0.6323,1,62.71
Chip Kelly,1165,1087,1125,498,1048,3958.0,0.5213,1,46.05
The Virtue Extollers,1801,1800,1759,112,1088,122.0,0.3604,0,36.40
The Dream Team,1146,1209,1164,434,1112,100.0,0.2798,1,34.58
Run CMC,837,811,838,796,693,52.0,0.3291,1,28.63
Naberhood Watch,93,87,84,1176,513,26.0,0.3089,1,15.24
ACE of L.A.D.,129,110,123,1223,934,1.0,0.1768,1,14.85
Commish-ion Impossible,690,695,697,926,256,8.0,0.2297,0,14.69
Maseratis Not Jerseys,10,14,6,509,915,0.0,0.1167,1,12.27


In [63]:
final.to_csv('OneUSAC_2025_playoffs_' + str(wk_count) + '.csv')

In [64]:
#table of distribution's of each team's simmed outcomes

outcome_dist = pd.pivot_table(array, values = ['place'], index = ['team'], columns=all_predictions.place.values, aggfunc='count', fill_value=0)

In [65]:
all_predictions['W2'] = [float(i) for i in array.W]
all_predictions['team'] = array.index
average_wins = pd.DataFrame(array.groupby('team')['W'].mean())
average_points = pd.DataFrame(array.groupby('team')['Points'].mean())

In [66]:
#season-ending placement probs

outcome_dist['Playoffs'] = (outcome_dist.place[1] + outcome_dist.place[2] 
                            + outcome_dist.place[3] + outcome_dist.place[4]
                            + outcome_dist.place[5] + outcome_dist.place[6]) / number_sims
outcome_dist['Consolation'] = (outcome_dist.place[7] + outcome_dist.place[8] 
                            + outcome_dist.place[9] + outcome_dist.place[10]
                            + outcome_dist.place[11] + outcome_dist.place[12]) / number_sims
outcome_dist['Bye'] = (outcome_dist.place[1] + outcome_dist.place[2]) / number_sims

outcome_dist['first'] = (outcome_dist.place[1]) / number_sims
outcome_dist['second'] = (outcome_dist.place[2]) / number_sims
outcome_dist['third'] = (outcome_dist.place[3]) / number_sims
outcome_dist['fourth'] = (outcome_dist.place[4]) / number_sims
outcome_dist['fifth'] = (outcome_dist.place[5]) / number_sims
outcome_dist['sixth'] = (outcome_dist.place[6]) / number_sims
outcome_dist['seventh'] = (outcome_dist.place[7]) / number_sims
outcome_dist['eighth'] = (outcome_dist.place[8]) / number_sims
outcome_dist['ninth'] = (outcome_dist.place[9]) / number_sims
outcome_dist['tenth'] = (outcome_dist.place[10]) / number_sims
outcome_dist['eleventh'] = (outcome_dist.place[11]) / number_sims
outcome_dist['twelfth'] = (outcome_dist.place[12]) / number_sims
outcome_dist['thirteenth'] = (outcome_dist.place[13]) / number_sims
outcome_dist['fourteenth'] = (outcome_dist.place[14]) / number_sims

outcome_dist = outcome_dist.sort_values(['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth','thirteenth','fourteenth'], ascending=False) #after 20
outcome2 = outcome_dist.drop(['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth','thirteenth','fourteenth'], axis=1) 

outcome2.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,'Playoffs','Consolation','Bye']

outcome3 = pd.merge(outcome2, average_wins, left_index=True, right_on='team')
outcome3 = pd.merge(outcome3, average_points, left_index=True, right_on='team')

outcome3.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,'Playoffs','Consolation','Bye','Wins','Points']
points_col = array.groupby('team').mean()
#outcome3['AvgPoints'] = round(points_col.Points, 2)

outcome3

outcome3.sort_values(['Playoffs'], ascending=False)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Playoffs,Consolation,Bye,Wins,Points
team,,,,,,,,,,,,,,,,,,,
The Abe W Memorial Champions,6223,2182,913,374,180,80,38,7,3,0,0,0,0,0,0.9952,0.0048,0.8405,10.2028,1596.272264
The Virtue Extollers,1813,2896,2075,1392,821,570,251,130,39,9,4,0,0,0,0.9567,0.0433,0.4709,9.4710,1499.981596
Kurtosis Warner,981,2135,2278,1742,1211,781,487,199,119,44,13,7,2,1,0.9128,0.0869,0.3116,8.4552,1611.303969
The Dream Team,389,982,1582,1840,1770,1418,961,552,276,155,59,12,4,0,0.7981,0.2015,0.1371,8.2709,1495.034870
Chip Kelly,376,1032,1528,1768,1592,1353,1015,615,423,170,74,42,11,1,0.7649,0.2339,0.1408,7.7166,1618.594753
Run CMC,129,489,907,1231,1619,1656,1437,1067,666,417,237,113,28,4,0.6031,0.3937,0.0618,7.5902,1483.359334
Commish-ion Impossible,83,254,558,1077,1483,1684,1579,1151,823,562,414,195,107,30,0.5139,0.4724,0.0337,7.5506,1434.420341
Rod the Great's Team,2,18,69,217,454,704,1034,1345,1307,1402,1237,1085,790,336,0.1464,0.7410,0.0020,6.3079,1393.462051
ACE of L.A.D.,4,8,60,139,297,488,815,1096,1355,1446,1463,1392,984,453,0.0996,0.7567,0.0012,5.8983,1415.041240


In [67]:
outcome3.sort_values(['Playoffs'], ascending=False).to_csv('OneUSAC2025_wk' + str(wk_count) + '.csv')

## Bonus: See R Code for viz output from this

Produces necessary tables for "progressive playoff odds" chart in rmd file

In [68]:
for_viz = pd.DataFrame(columns=['team', 'Playoffs', 'Consolation', 'Bye', 'week'])

for i in range(0,wk_count+1):
    path = 'OneUSAC2025_wk' + str(i) + '.csv'
    numbers = pd.read_csv(path)
    numbers = numbers[['team', 'Playoffs', 'Consolation', 'Bye']]
    numbers['week'] = i
    for_viz = pd.concat([for_viz, numbers], ignore_index=True)

for_viz.to_csv('spray_chart.csv')
for_viz.to_csv('weekly_chart' + str(wk_count) + '.csv')

C:\Users\zachary.gozlan\AppData\Local\Temp\ipykernel_13040\4053141938.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  for_viz = pd.concat([for_viz, numbers], ignore_index=True)


In [69]:
save_grief = 'tmp$team <- factor(tmp$team, c("'

for i in for_viz[for_viz.week == wk_count]['team']:
    save_grief += i
    save_grief += '","'
    
save_grief = save_grief[:-3]
save_grief += '"))'

save_grief

'tmp$team <- factor(tmp$team, c("The Abe W Memorial Champions","The Virtue Extollers","Kurtosis Warner","The Dream Team","Chip Kelly","Run CMC","Commish-ion Impossible","Rod the Great\'s Team","ACE of L.A.D.","Naberhood Watch","Takoma Queries (ex-Abe)","#CorruptCommish","The Terrible Towels","Maseratis Not Jerseys"))'

In [70]:
for_viz[for_viz.week == wk_count]

,team,Playoffs,Consolation,Bye,week
141,The Abe W Memorial Champions,0.9952,0.0048,0.8405,9
142,The Virtue Extollers,0.9567,0.0433,0.4709,9
143,Kurtosis Warner,0.9128,0.0869,0.3116,9
144,The Dream Team,0.7981,0.2015,0.1371,9
145,Chip Kelly,0.7649,0.2339,0.1408,9
146,Run CMC,0.6031,0.3937,0.0618,9
147,Commish-ion Impossible,0.5139,0.4724,0.0337,9
148,Rod the Great's Team,0.1464,0.7410,0.0020,9
149,ACE of L.A.D.,0.0996,0.7567,0.0012,9
150,Naberhood Watch,0.0734,0.7467,0.0002,9


In [72]:
try as stop

#nonsense code to break the script


SyntaxError: expected ':' (344124135.py, line 1)

## Trade Assistance

Has worked in the past but likely deprecated now

In [ ]:
breakdown = pd.DataFrame(columns=['Week', 'Team', 'QB', 'RB', 'FlexRB', 'WR', 'FlexWR', 'TE', 'FlexTE', 'Def', 'Kicker'])

In [ ]:
for week in range(5,wk_count+1):
    for team in range(1,15): 
        time.sleep(1)
        
        url = 'https://football.fantasysports.yahoo.com/f1/664138/matchup?week=' + str(week) + '&mid1=' + str(team) + '&mid2=7'

        driver.get(url)
        
        if team == 7:
            team2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/section[1]/div/div[1]/div[1]/div[1]/div[2]/div[1]/a').text

            qb2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[1]/td[4]/div/a').text)
            wr2a = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[2]/td[4]/div/a').text)
            wr2b = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[3]/td[4]/div/a').text)
            rb2a = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[4]/td[4]/div/a').text)
            rb2b = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[5]/td[4]/div/a').text)
            te2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[6]/td[4]/div/a').text)
            flex2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[7]/td[4]/div/a').text)
            k2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[8]/td[4]/div/a').text)
            defense2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[9]/td[4]/div/a').text)

            flex_pos1 = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[7]/td[2]/div/div/div[1]/div/span').text[-2:]
            
            if flex_pos2 == 'RB':
                newrow = {'Week': week, 'Team': team2, 'QB': qb2, 'RB': rb2a + rb2b, 'WR': wr2a + wr2b, 'TE': te2, \
                         'FlexRB': flex2, 'Def': defense2, 'Kicker': k2}
                breakdown =  breakdown.append(newrow, ignore_index = True)
            if flex_pos2 == 'WR':
                newrow = {'Week': week, 'Team': team2, 'QB': qb2, 'RB': rb2a + rb2b, 'WR': wr2a + wr2b, 'TE': te2, \
                         'FlexWR': flex2, 'Def': defense2, 'Kicker': k2}
                breakdown =  breakdown.append(newrow, ignore_index = True)
            if flex_pos2 == 'TE':
                newrow = {'Week': week, 'Team': team2, 'QB': qb2, 'RB': rb2a + rb2b, 'WR': wr2a + wr2b, 'TE': te2, \
                         'FlexTE': flex2, 'Def': defense2, 'Kicker': k2}
                breakdown =  breakdown.append(newrow, ignore_index = True)
            
            continue 
            
        team2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/section[1]/div/div[1]/div[3]/div[1]/div[2]/div[1]/a').text

        qb2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[1]/td[8]/div/a').text)
        wr2a = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[2]/td[8]/div/a').text)
        wr2b = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[3]/td[8]/div/a').text)
        rb2a = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[4]/td[8]/div/a').text)
        rb2b = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[5]/td[8]/div/a').text)
        te2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[6]/td[8]/div/a').text)
        flex2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[7]/td[8]/div/a').text)
        k2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[8]/td[8]/div/a').text)
        defense2 = float(driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[9]/td[8]/div/a').text)

        flex_pos2 = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/div[1]/div[1]/div/table/tbody/tr[7]/td[10]/div/div/div[1]/div/span').text[-2:]

        '''newrow = {'Week': 1, 'Team': team1, 'QB': qb1, 'RB': rb1a + rb1b, 'WR': wr1a + wr1b, 'TE': te1, \
                 'Flex': flex1, 'FlexPos': flex_pos1, 'Def': defense1, 'Kicker': k1}
        breakdown =  breakdown.append(newrow, ignore_index = True)'''
        
        if flex_pos2 == 'RB':
            newrow = {'Week': week, 'Team': team2, 'QB': qb2, 'RB': rb2a + rb2b, 'WR': wr2a + wr2b, 'TE': te2, \
                     'FlexRB': flex2, 'Def': defense2, 'Kicker': k2}
            breakdown =  breakdown.append(newrow, ignore_index = True)
        if flex_pos2 == 'WR':
            newrow = {'Week': week, 'Team': team2, 'QB': qb2, 'RB': rb2a + rb2b, 'WR': wr2a + wr2b, 'TE': te2, \
                     'FlexWR': flex2, 'Def': defense2, 'Kicker': k2}
            breakdown =  breakdown.append(newrow, ignore_index = True)
        if flex_pos2 == 'TE':
            newrow = {'Week': week, 'Team': team2, 'QB': qb2, 'RB': rb2a + rb2b, 'WR': wr2a + wr2b, 'TE': te2, \
                     'FlexTE': flex2, 'Def': defense2, 'Kicker': k2}
            breakdown =  breakdown.append(newrow, ignore_index = True)

In [ ]:
breakdown.to_csv('position_points_' +str(wk_count) +'.csv')